In [1]:
import pandas as pd
import numpy as np

**Reading Transactions File**

In [2]:
df_txn=pd.read_csv('transactions.csv')

In [3]:
df_txn.head()

,doj,doi,srcid,destid,srcid_region,destid_region,srcid_tier,destid_tier,cumsum_seatcount,cumsum_searchcount,dbd
0,2023-03-01,2023-01-30,45,46,Karnataka,Tamil Nadu,Tier 1,Tier 1,8.0,76.0,30
1,2023-03-01,2023-01-30,46,45,Tamil Nadu,Karnataka,Tier 1,Tier 1,8.0,70.0,30
2,2023-03-01,2023-01-30,45,47,Karnataka,Andhra Pradesh,Tier 1,Tier 1,4.0,142.0,30
3,2023-03-01,2023-01-30,47,45,Andhra Pradesh,Karnataka,Tier 1,Tier 1,0.0,68.0,30
4,2023-03-01,2023-01-30,46,9,Tamil Nadu,Tamil Nadu,Tier 1,Tier2,9.0,162.0,30


**Filtering Out DBD=15 since we want to predict bookings 15 days prior to booking**

In [4]:
df_txn=df_txn[df_txn['dbd']==15]

In [5]:
df_txn.head()

,doj,doi,srcid,destid,srcid_region,destid_region,srcid_tier,destid_tier,cumsum_seatcount,cumsum_searchcount,dbd
1500,2023-03-01,2023-02-14,45,46,Karnataka,Tamil Nadu,Tier 1,Tier 1,16.0,480.0,15
1501,2023-03-01,2023-02-14,46,45,Tamil Nadu,Karnataka,Tier 1,Tier 1,34.0,352.0,15
1502,2023-03-01,2023-02-14,45,47,Karnataka,Andhra Pradesh,Tier 1,Tier 1,36.0,892.0,15
1503,2023-03-01,2023-02-14,47,45,Andhra Pradesh,Karnataka,Tier 1,Tier 1,18.0,1130.0,15
1504,2023-03-01,2023-02-14,46,9,Tamil Nadu,Tamil Nadu,Tier 1,Tier2,48.0,1023.0,15


In [6]:
df_txn.info()

<class 'pandas.core.frame.DataFrame'>
Index: 73100 entries, 1500 to 2264599
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   doj                 73100 non-null  object 
 1   doi                 73100 non-null  object 
 2   srcid               73100 non-null  int64  
 3   destid              73100 non-null  int64  
 4   srcid_region        73100 non-null  object 
 5   destid_region       73100 non-null  object 
 6   srcid_tier          73100 non-null  object 
 7   destid_tier         73100 non-null  object 
 8   cumsum_seatcount    73100 non-null  float64
 9   cumsum_searchcount  73100 non-null  float64
 10  dbd                 73100 non-null  int64  
dtypes: float64(2), int64(3), object(6)
memory usage: 8.7+ MB


In [7]:
df_txn['doj']=pd.to_datetime(df_txn['doj'])
df_txn['doi']=pd.to_datetime(df_txn['doi'])

In [8]:
for col in ['srcid', 'destid','srcid_region','destid_region','srcid_tier','destid_tier']:
    df_txn[col] = df_txn[col].astype('category')

In [9]:
df_txn.info()

<class 'pandas.core.frame.DataFrame'>
Index: 73100 entries, 1500 to 2264599
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   doj                 73100 non-null  datetime64[ns]
 1   doi                 73100 non-null  datetime64[ns]
 2   srcid               73100 non-null  category      
 3   destid              73100 non-null  category      
 4   srcid_region        73100 non-null  category      
 5   destid_region       73100 non-null  category      
 6   srcid_tier          73100 non-null  category      
 7   destid_tier         73100 non-null  category      
 8   cumsum_seatcount    73100 non-null  float64       
 9   cumsum_searchcount  73100 non-null  float64       
 10  dbd                 73100 non-null  int64         
dtypes: category(6), datetime64[ns](2), float64(2), int64(1)
memory usage: 5.8 MB


In [10]:
df_txn.describe()

,doj,doi,cumsum_seatcount,cumsum_searchcount,dbd
count,73100,73100,73100.000000,73100.000000,73100.0
mean,2024-02-28 23:59:59.999999744,2024-02-13 23:59:59.999999744,54.444241,1948.652298,15.0
min,2023-03-01 00:00:00,2023-02-14 00:00:00,0.000000,0.000000,15.0
25%,2023-08-30 00:00:00,2023-08-15 00:00:00,4.000000,348.000000,15.0
50%,2024-02-29 00:00:00,2024-02-14 00:00:00,16.000000,780.000000,15.0
75%,2024-08-30 00:00:00,2024-08-15 00:00:00,51.000000,1684.000000,15.0
max,2025-02-28 00:00:00,2025-02-13 00:00:00,4724.000000,391464.000000,15.0
std,NaN,NaN,135.714726,6834.956089,0.0


In [11]:
df_txn.rename(columns={
    'cumsum_seatcount': 'bookings_dbd_15',
    'cumsum_searchcount': 'searches_dbd_15'
}, inplace=True)

**Dropping dbd column as it is redundant now**

In [12]:
df_txn.drop(columns=['dbd'], inplace=True)

**Reading the Train File**

In [13]:
df_train=pd.read_csv('train.csv')

In [14]:
df_train.head()

,doj,srcid,destid,final_seatcount
0,2023-03-01,45,46,2838.0
1,2023-03-01,46,45,2298.0
2,2023-03-01,45,47,2720.0
3,2023-03-01,47,45,2580.0
4,2023-03-01,46,9,4185.0


In [15]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67200 entries, 0 to 67199
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   doj              67200 non-null  object 
 1   srcid            67200 non-null  int64  
 2   destid           67200 non-null  int64  
 3   final_seatcount  67200 non-null  float64
dtypes: float64(1), int64(2), object(1)
memory usage: 2.1+ MB


In [16]:
len(df_train)

67200

In [17]:
len(df_txn)

73100

In [18]:
for col in ['srcid', 'destid']:
    df_train[col] = df_train[col].astype('category')

In [19]:
df_train['doj']=pd.to_datetime(df_train['doj'])

In [20]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67200 entries, 0 to 67199
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   doj              67200 non-null  datetime64[ns]
 1   srcid            67200 non-null  category      
 2   destid           67200 non-null  category      
 3   final_seatcount  67200 non-null  float64       
dtypes: category(2), datetime64[ns](1), float64(1)
memory usage: 1.2 MB


In [21]:
df_train.describe()

,doj,final_seatcount
count,67200,67200.000000
mean,2024-01-30 11:59:59.999999744,2001.729464
min,2023-03-01 00:00:00,2.000000
25%,2023-08-15 18:00:00,1252.000000
50%,2024-01-30 12:00:00,1685.000000
75%,2024-07-16 06:00:00,2408.000000
max,2024-12-31 00:00:00,13503.000000
std,NaN,1194.711140


In [22]:
df_test=pd.read_csv('test_redbus.csv')

In [23]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5900 entries, 0 to 5899
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   route_key  5900 non-null   object
 1   doj        5900 non-null   object
 2   srcid      5900 non-null   int64 
 3   destid     5900 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 184.5+ KB


In [24]:
df_test['doj']=pd.to_datetime(df_test['doj'])

In [25]:
for col in ['srcid', 'destid']:
    df_test[col] = df_test[col].astype('category')

In [26]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5900 entries, 0 to 5899
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   route_key  5900 non-null   object        
 1   doj        5900 non-null   datetime64[ns]
 2   srcid      5900 non-null   category      
 3   destid     5900 non-null   category      
dtypes: category(2), datetime64[ns](1), object(1)
memory usage: 106.6+ KB


In [27]:
df_test.describe()

,doj
count,5900
mean,2025-01-30 00:00:00
min,2025-01-01 00:00:00
25%,2025-01-15 00:00:00
50%,2025-01-30 00:00:00
75%,2025-02-14 00:00:00
max,2025-02-28 00:00:00


**Merging train and test data with transaction data**

In [28]:
# Merge with training data
df_train_merged = df_train.merge(
    df_txn, on=['srcid', 'destid', 'doj'], how='left'
)

# Merge with test data
df_test_merged = df_test.merge(
    df_txn, on=['srcid', 'destid', 'doj'], how='left'
)


In [29]:
route_counts = df_train_merged.groupby(['srcid', 'destid']).size().reset_index(name='route_freq')

df_train_merged = df_train_merged.merge(route_counts, on=['srcid', 'destid'], how='left')
df_test_merged = df_test_merged.merge(route_counts, on=['srcid', 'destid'], how='left')

# Force route_freq to be numeric and fill NaN only in test
df_test_merged['route_freq'] = pd.to_numeric(df_test_merged['route_freq'], errors='coerce').fillna(0)


/tmp/ipython-input-29-2466220207.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  route_counts = df_train_merged.groupby(['srcid', 'destid']).size().reset_index(name='route_freq')


In [30]:
df_train_merged['is_intra_state'] = (df_train_merged['srcid_region'] == df_train_merged['destid_region']).astype(int)
df_test_merged['is_intra_state'] = (df_test_merged['srcid_region'] == df_test_merged['destid_region']).astype(int)


In [31]:
# Sanity check
print("Train shape:", df_train_merged.shape)
print("Test shape:", df_test_merged.shape)

# Check for missing txn info (e.g., no dbd=15 snapshot)
print("Missing in train:", df_train_merged.isna().sum())
print("Missing in test:", df_test_merged.isna().sum())


Train shape: (67200, 13)
Test shape: (5900, 13)
Missing in train: doj                0
srcid              0
destid             0
final_seatcount    0
doi                0
srcid_region       0
destid_region      0
srcid_tier         0
destid_tier        0
bookings_dbd_15    0
searches_dbd_15    0
route_freq         0
is_intra_state     0
dtype: int64
Missing in test: route_key          0
doj                0
srcid              0
destid             0
doi                0
srcid_region       0
destid_region      0
srcid_tier         0
destid_tier        0
bookings_dbd_15    0
searches_dbd_15    0
route_freq         0
is_intra_state     0
dtype: int64


In [32]:
df_train_merged.head()

,doj,srcid,destid,final_seatcount,doi,srcid_region,destid_region,srcid_tier,destid_tier,bookings_dbd_15,searches_dbd_15,route_freq,is_intra_state
0,2023-03-01,45,46,2838.0,2023-02-14,Karnataka,Tamil Nadu,Tier 1,Tier 1,16.0,480.0,672,0
1,2023-03-01,46,45,2298.0,2023-02-14,Tamil Nadu,Karnataka,Tier 1,Tier 1,34.0,352.0,672,0
2,2023-03-01,45,47,2720.0,2023-02-14,Karnataka,Andhra Pradesh,Tier 1,Tier 1,36.0,892.0,672,0
3,2023-03-01,47,45,2580.0,2023-02-14,Andhra Pradesh,Karnataka,Tier 1,Tier 1,18.0,1130.0,672,0
4,2023-03-01,46,9,4185.0,2023-02-14,Tamil Nadu,Tamil Nadu,Tier 1,Tier2,48.0,1023.0,672,1


**Feature Engineering**

In [33]:
# Create weekday (0 = Monday, 6 = Sunday)
df_train_merged['weekday_num'] = df_train['doj'].dt.dayofweek
df_test_merged['weekday_num'] = df_test['doj'].dt.dayofweek

In [34]:
df_train_merged.head()


,doj,srcid,destid,final_seatcount,doi,srcid_region,destid_region,srcid_tier,destid_tier,bookings_dbd_15,searches_dbd_15,route_freq,is_intra_state,weekday_num
0,2023-03-01,45,46,2838.0,2023-02-14,Karnataka,Tamil Nadu,Tier 1,Tier 1,16.0,480.0,672,0,2
1,2023-03-01,46,45,2298.0,2023-02-14,Tamil Nadu,Karnataka,Tier 1,Tier 1,34.0,352.0,672,0,2
2,2023-03-01,45,47,2720.0,2023-02-14,Karnataka,Andhra Pradesh,Tier 1,Tier 1,36.0,892.0,672,0,2
3,2023-03-01,47,45,2580.0,2023-02-14,Andhra Pradesh,Karnataka,Tier 1,Tier 1,18.0,1130.0,672,0,2
4,2023-03-01,46,9,4185.0,2023-02-14,Tamil Nadu,Tamil Nadu,Tier 1,Tier2,48.0,1023.0,672,1,2


In [46]:
df_test_merged.head()

,route_key,doj,srcid,destid,doi,srcid_region,destid_region,srcid_tier,destid_tier,bookings_dbd_15,searches_dbd_15,route_freq,is_intra_state,weekday_num,search_to_book_ratio,bookings_x_weekday
0,2025-02-11_46_45,2025-02-11,46,45,2025-01-27,Tamil Nadu,Karnataka,Tier 1,Tier 1,38.0,1082.0,672,0,1,28.473684,38.0
1,2025-01-20_17_23,2025-01-20,17,23,2025-01-05,East 1,East 1,Tier2,Tier 1,0.0,1175.0,672,1,0,11750.000000,0.0
2,2025-01-08_02_14,2025-01-08,2,14,2024-12-24,Maharashtra and Goa,Maharashtra and Goa,Tier 1,Tier2,0.0,370.0,672,1,2,3700.000000,0.0
3,2025-01-08_08_47,2025-01-08,8,47,2024-12-24,Andhra Pradesh,Andhra Pradesh,Tier2,Tier 1,0.0,120.0,672,1,2,1200.000000,0.0
4,2025-01-08_09_46,2025-01-08,9,46,2024-12-24,Tamil Nadu,Tamil Nadu,Tier2,Tier 1,39.0,1230.0,672,1,2,31.538462,78.0


**Creating A New Variable Called Search/Bookings Ratio to measure demand**

Rows having 0 searches have been estimated to 0.1 to facilitate division

In [36]:
len(df_train_merged[df_train_merged['bookings_dbd_15']==0])

15756

In [37]:
df_train_merged['search_to_book_ratio'] = (
    df_train_merged['searches_dbd_15'] / df_train_merged['bookings_dbd_15'].replace(0, 0.1))

In [38]:
len(df_test_merged[df_test_merged['bookings_dbd_15']==0])

995

In [39]:
df_test_merged['search_to_book_ratio'] = (
    df_test_merged['searches_dbd_15'] / df_test_merged['bookings_dbd_15'].replace(0, 0.1))

In [44]:
df_train_merged['bookings_x_weekday'] = df_train_merged['bookings_dbd_15'] * df_train_merged['weekday_num']
df_test_merged['bookings_x_weekday'] = df_test_merged['bookings_dbd_15'] * df_test_merged['weekday_num']


In [45]:
df_train_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67200 entries, 0 to 67199
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   doj                   67200 non-null  datetime64[ns]
 1   srcid                 67200 non-null  category      
 2   destid                67200 non-null  category      
 3   final_seatcount       67200 non-null  float64       
 4   doi                   67200 non-null  datetime64[ns]
 5   srcid_region          67200 non-null  category      
 6   destid_region         67200 non-null  category      
 7   srcid_tier            67200 non-null  category      
 8   destid_tier           67200 non-null  category      
 9   bookings_dbd_15       67200 non-null  float64       
 10  searches_dbd_15       67200 non-null  float64       
 11  route_freq            67200 non-null  int64         
 12  is_intra_state        67200 non-null  int64         
 13  weekday_num     

**Data Modelling**

In [47]:
!pip install optuna
import pandas as pd
import numpy as np
from lightgbm import LGBMRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import optuna

In [48]:
from lightgbm import LGBMRegressor, early_stopping, log_evaluation
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import optuna

# Feature set
features = [
    'srcid', 'destid',
    'srcid_region', 'destid_region',
    'srcid_tier', 'destid_tier',
    'bookings_dbd_15', 'searches_dbd_15',
    'search_to_book_ratio', 'weekday_num',
    'route_freq', 'is_intra_state'
]

X = df_train_merged[features]
y = df_train_merged['final_seatcount']
X_test = df_test_merged[features]

cat_features = [col for col in features if str(X[col].dtype) == 'category']

# Optuna objective
def objective(trial):
    params = {
        'objective': 'regression',
        'metric': 'rmse',
        'verbosity': -1,
        'boosting_type': 'gbdt',
        'learning_rate': trial.suggest_float("learning_rate", 0.01, 0.1),
        'num_leaves': trial.suggest_int("num_leaves", 20, 100),
        'max_depth': trial.suggest_int("max_depth", 3, 10),
        'feature_fraction': trial.suggest_float("feature_fraction", 0.6, 1.0),
        'bagging_fraction': trial.suggest_float("bagging_fraction", 0.6, 1.0),
        'bagging_freq': trial.suggest_int("bagging_freq", 1, 7),
        'min_child_samples': trial.suggest_int("min_child_samples", 10, 100)
    }

    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    rmse_scores = []

    for train_idx, val_idx in kf.split(X):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

        model = LGBMRegressor(**params)
        model.fit(
            X_train, y_train,
            eval_set=[(X_val, y_val)],
            eval_metric="rmse",
            categorical_feature=cat_features,
            callbacks=[
                early_stopping(stopping_rounds=50),
                log_evaluation(0)
            ]
        )
        preds = model.predict(X_val)
        rmse = np.sqrt(mean_squared_error(y_val, preds))
        rmse_scores.append(rmse)

    return np.mean(rmse_scores)

# Run optimization
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=30)

# Train final model
best_params = study.best_params
best_params.update({'objective': 'regression', 'metric': 'rmse'})

final_model = LGBMRegressor(**best_params)
final_model.fit(X, y, categorical_feature=cat_features)

# Predict
test_preds = final_model.predict(X_test)

# Submission
submission = df_test_merged[['route_key']].copy()
submission['final_seatcount'] = test_preds
submission.to_csv("submission_file.csv", index=False)
print(" submission_file.csv created")


[I 2025-06-21 03:10:20,473] A new study created in memory with name: no-name-c5953917-770a-4637-888f-2b7a640c27e8


Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's rmse: 571.306
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's rmse: 571.217
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's rmse: 566.985
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's rmse: 571.913
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's rmse: 563.464


[I 2025-06-21 03:10:37,321] Trial 0 finished with value: 568.9768225095546 and parameters: {'learning_rate': 0.01951734440842117, 'num_leaves': 76, 'max_depth': 7, 'feature_fraction': 0.764961087353285, 'bagging_fraction': 0.7584002799193388, 'bagging_freq': 3, 'min_child_samples': 17}. Best is trial 0 with value: 568.9768225095546.


Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's rmse: 698.169
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's rmse: 698.263
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's rmse: 696.434
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's rmse: 694.114
Training until validation scores don't improve for 50 rounds


[I 2025-06-21 03:10:44,931] Trial 1 finished with value: 694.5841177751776 and parameters: {'learning_rate': 0.014340508119641979, 'num_leaves': 70, 'max_depth': 5, 'feature_fraction': 0.6518649386492948, 'bagging_fraction': 0.8220635730778151, 'bagging_freq': 7, 'min_child_samples': 26}. Best is trial 0 with value: 568.9768225095546.


...

Did not meet early stopping. Best iteration is:
[100]	valid_0's rmse: 489.455
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's rmse: 475.036
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's rmse: 471.457
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's rmse: 464.687
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[99]	valid_0's rmse: 473.087
Training until validation scores don't improve for 50 rounds


[I 2025-06-21 03:13:38,541] Trial 28 finished with value: 470.62396696785726 and parameters: {'learning_rate': 0.08380122571726926, 'num_leaves': 58, 'max_depth': 9, 'feature_fraction': 0.9992380071111908, 'bagging_fraction': 0.7384768730780258, 'bagging_freq': 4, 'min_child_samples': 21}. Best is trial 10 with value: 469.74428872601436.


Did not meet early stopping. Best iteration is:
[98]	valid_0's rmse: 468.852
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's rmse: 477.361
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's rmse: 471.366
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's rmse: 468.804
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's rmse: 473.949
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's rmse: 474.1


[I 2025-06-21 03:13:45,806] Trial 29 finished with value: 473.1158531762079 and parameters: {'learning_rate': 0.07163725469903033, 'num_leaves': 68, 'max_depth': 7, 'feature_fraction': 0.7810552548862809, 'bagging_fraction': 0.7740247289865786, 'bagging_freq': 3, 'min_child_samples': 51}. Best is trial 10 with value: 469.74428872601436.


 submission_file.csv created


In [49]:
# Evaluate final model performance on training data (since we don't have test labels)
train_preds = final_model.predict(X)
rmse_final = np.sqrt(mean_squared_error(y, train_preds))

print(f" Final model training RMSE: {rmse_final:.2f}")


 Final model training RMSE: 440.33
